In [52]:
import boto3
import networkx as nx
import numpy as np
import uuid
import time

import os
import io
import configparser

In [32]:
config = configparser.ConfigParser()

# Write example config to disk.
config['AWS'] = {
    'ACCESS_KEY': '',
    'SECRET_KEY': '',
    'SESSION_TOKEN': ''
}

config['Master'] = {
    'MASTER_URL': '',
    'API_KEY': ''
}

with open('config.txt.sample', 'w') as configfile:
    config.write(configfile)
    
config = configparser.ConfigParser()
config.read('config.txt')

session = boto3.Session(
    aws_access_key_id=config['AWS']['access_key'], 
    aws_secret_access_key=config['AWS']['secret_key'], 
)


In [58]:
def getInstanceIdsByType():
    response = session.client("ec2").describe_instances()
    reservations = response['Reservations']
    
    instancesByType = dict()
    for reservation in reservations:
        for instance in reservation['Instances']:
            for tag in instance['Tags']:
                if tag['Key'] == 'instance_type':
                    instancetype = tag['Value']
                    if not instancetype in instancesByType:
                        instancesByType[instancetype] = [instance['InstanceId']]
                    else:
                        instancesByType[instancetype].append(instance['InstanceId'])
    
    return instancesByType    

def terminateInstancesWithIds(ids):
    session.client("ec2").terminate_instances(InstanceIds=ids)

def createMaster(session):
    ec2_client = session.client('ec2')

def generate_graph_file(directory, n, p, prepend_vertex_weights=False):
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    file = os.path.join(directory, '%s-%d-%s.graph' % (uuid.uuid4(), n, p))

    # Generate ER graph with n nodes and edge probability p
    G = nx.erdos_renyi_graph(int(n), p)
    node_weights = []

    # Generate random edge weights.
    for (u, v) in G.edges():
        G.edges[u,v]['weight'] = round(np.random.random(), 2)

    nx.write_weighted_edgelist(G, file, delimiter=",")

    if prepend_vertex_weights:
        # Prepend weights
        for u in G.nodes():
            node_weights.append(str(1.0 / n)) # Uniform for pagerank.

        with open(file, 'r') as original: data = original.read()
        with open(file, 'w') as modified: modified.write(",".join(node_weights) + "\n" + data)
            
    return file

def generate_graph(output, n, p, prepend_vertex_weight=False):    
    if prepend_vertex_weight:
        # Uniform for pagerank.
        node_weights = [str(1.0 / n)] * n
        output.write(output.write(",".join(node_weights) + "\n"))

    # Generate ER graph with n nodes and edge probability p
    G = nx.erdos_renyi_graph(int(n), p)
    
    # Generate random edge weights.
    for (u, v) in G.edges():
        G.edges[u,v]['weight'] = round(np.random.random(), 2)
        
    node_weights = []

    nx.write_weighted_edgelist(G, output, delimiter=",")

def run_experiment():
    
    try:
        # Connect to EC2
        ec2_client = session.client('ec2')

        # Spin up a master.
        response = ec2_client.run_instances(LaunchTemplate={'LaunchTemplateName': "Master"}, MinCount=1, MaxCount=1)

        initialized = False
        # Wait until the master is ready.
        while not initialized:
            # Every second attempt to contact /health
            time.sleep(1)

            # If /health returns we are ready.
            initialized=True

        # Generate graphs and post them to the system.
        for i in range(1000):
            output = io.BytesIO()
            generate_graph(output, 10, 0.5)
            graph = output.getvalue().decode('utf-8')

            # Post to system.

        # If all graphs are posted we start polling every 10 seconds if the system is done.
        finished = False
        while not finished:
            # Every 10 seconds, check if we're done.
            time.sleep(10)
            finished = True
            
        # Now make sure all logs are written and all workers are being killed.
    finally:
        # Sleep so the master can finish/kill workers.
        time.sleep(10)
        
        # Terminate all masters.
        instances = getInstanceIdsByType()
        terminateInstancesWithIds(instances['master'])

In [62]:
#run_experiment()

'0,2,0.28\n0,6,0.51\n0,9,0.86\n1,2,0.61\n1,4,0.79\n1,7,0.27\n2,3,0.75\n2,4,0.01\n3,4,0.1\n3,5,0.95\n3,6,0.96\n3,8,0.49\n4,6,0.04\n4,7,0.63\n4,8,0.65\n5,7,0.07\n5,8,0.49\n6,7,0.32\n6,8,0.17\n7,8,0.85\n7,9,0.39\n'

In [49]:
# directory = 'graphs'
# node = [1e1, 1e2, 1e3]
# p_edge = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

# # For each node_vertex combination
# for n in node:
#     for p in p_edge:
#         generate_graph_file(directory, n, p, prepend_vertex_weights=True)